## Maximum likelihood model fitting

In [1]:
import numpy as np
import os 
import arviz as az
import scipy as sp
import scipy.io as sio
import scipy.stats as stats
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pickle
import importlib
import math
import xarray as xr

In [2]:
os.chdir('..\\Pystan\\nc_files')

In [3]:
standard_basic_fit = az.from_netcdf('standard_basic_fit.nc')
uncued_basic_fit = az.from_netcdf('uncued_basic_fit.nc')

In [4]:
uncued_basic_fit

Inference data with groups:
	> posterior
	> sample_stats

In [ ]:
sbf_df = standard_basic_fit.posterior.beta.to_dataframe()
sbf_df_etaP = standard_basic_fit.posterior.etaPositive.to_dataframe()
sbf_df_etaP

In [ ]:
beta_values = sbf_df[sbf_df.index.get_level_values('beta_dim_0').isin([0])]

In [ ]:
etaP_values = sbf_df_etaP[sbf_df_etaP.index.get_level_values('etaPositive_dim_0').isin([0])]

In [ ]:
np.concatenate([beta_values, etaP_values], axis = 1)

In [5]:
def get_params_basic_4_chains(fit, s):
    """takes in fit, s (subject number), and outputs params_s using ALL CHAINS
    function is called basic due to the parameters chosen"""
    
    fit_beta_df = fit.posterior.beta.to_dataframe()
    fit_etaP_df = fit.posterior.etaPositive.to_dataframe()
    fit_etaN_df = fit.posterior.etaNegative.to_dataframe()
    
    params_s = np.concatenate([fit_beta_df[fit_beta_df.index.get_level_values('beta_dim_0').isin([s])],
                              fit_etaP_df[fit_etaP_df.index.get_level_values('etaPositive_dim_0').isin([s])],
                              fit_etaN_df[fit_etaN_df.index.get_level_values('etaNegative_dim_0').isin([s])]], 
                              axis = 1)
    return params_s

In [ ]:
params_all_4 = get_params_basic_4_chains(standard_basic_fit, 0)
params_all_4

In [ ]:
standard_basic_fit.posterior.beta #beta's DataArray 
# standard_basic_fit.posterior.beta.beta_dim_0 #beta's subjects
standard_basic_fit.posterior.beta[0] #first chain, storing an array of beta values
chain_1_beta = standard_basic_fit.posterior.beta[0] #first chain, each column represents a subject; each column stores the posterior values of beta
chain_1_beta_df = pd.DataFrame(data = chain_1_beta) #chain_1 as a df
chain_1_beta[:,0] #first column of the first chain, representing the beta values of the posterior 

In [ ]:
chain_1_beta

In [ ]:
standard_basic_fit.posterior.etaPositive #etaPositive's DataArray
standard_basic_fit.posterior.etaPositive[0] #first chain
chain_1_etaP = standard_basic_fit.posterior.etaPositive[0]

standard_basic_fit.posterior.etaNegative #etaNegative's DataArray
standard_basic_fit.posterior.etaNegative[0] #first chain
chain_1_etaN = standard_basic_fit.posterior.etaNegative[0]

standard_basic_fit.posterior.m #m's DataArray 
standard_basic_fit.posterior.m[0] #first chain
chain_1_m = standard_basic_fit.posterior.m[0]

In [ ]:
chain_1_etaP
chain_1_etaN
chain_1_m

In [ ]:
chain_1_etaP[:,[0]]
chain_1_etaN[:,[0]]
chain_1_m[:,[0]]

## Goal: create params for one subject (params_s)

params_s will be 4 columns (representing beta, etaP, etaN and m) and 1000 rows (representing the 1000 draws)

## Goal 2: write for loop such that the code can run multiple times, using a different subject each time ***
- This will involve taking the first column from each parameter's dataarray (for subject 0), appending them (side-by-side), then running the code

In [ ]:
array1 = np.array([[1, 1, 1], [2, 2, 2]])
array2 = np.array([[3, 3, 3], [4, 4, 4]])

appended = np.concatenate([array1, array2], axis = 1)
appended

In [ ]:
array1 = np.array([[1,2,3]])
array2 = np.array([[4,5,6]])

appended = np.concatenate([array1, array2], axis = 0)
# appended

In [ ]:
parameters = np.concatenate([chain_1_beta[:,[0,1]], chain_1_etaP[:,[0,1]], chain_1_etaN[:,[0,1]]], axis = 1)
# parameters #a workaround, but contains 2 beta values, 2 etaP, and 2etaN (as the 6 columns)

In [ ]:
params_1 = np.concatenate([chain_1_beta[:,[0]], chain_1_etaP[:,[0]], chain_1_etaN[:,[0]], chain_1_m[:,[0]]], axis = 1) #extra square parentheses add a dimension
params_1

In [ ]:
def get_params_basic(fit, s):
    """takes in fit, s (subject number), and outputs params_s using the first chain
    function is called basic due to the parameters chosen"""
    params_s = np.concatenate([fit.posterior.beta[0][:,[s-1]], 
                               fit.posterior.etaPositive[0][:,[s-1]], 
                               fit.posterior.etaNegative[0][:,[s-1]]], 
                              axis = 1)
    return params_s

#fit.posterior.m[0][:,[s-1]]

In [ ]:
params_1 = get_params_basic(standard_basic_fit, 1)
params_1

In [ ]:
def get_params(fit, subs): 
    """gets params for subs, this function is not functional yet"""
    for s in range(subs):
        params = np.concatenate([fit.posterior.beta[:,[s]], fit.posterior.etaPositive[:,[s]], fit.posterior.etaNegative[:,[s]]], axis = 1)
        #... rest of body 
    return params

In [ ]:
# get_params(standard_basic_fit, 1)

# Original simulation code

In [ ]:
np.random.seed(
ntr = 200 # num trials #might be 1000

def mysimulation(params,ntr):
    
    # params is a 3 vector of beta, etaP, etaN
    
    V = np.zeros(4) # each option has a value
    beta = params[0]
    etaP = params[1]
    etaN = params[2]
    decay = params[3] # decay between zero and 1
    
#     V[3] = 4
    
    p_win = [0.9,0.8,0.5,0.4]
    win_amount = [1,2,3,4]
    pun_dur = [5,10,30,40]
    
    Q = np.zeros([4,ntr])
    choice = []
    win = []
    probs = []
    
    
    for t in range(ntr):
#         print(t)

        Q[:,t] = V
        
        # now we want to calculate the log likelihood of the choice on the current trial
        # we assume the prob of each choice follows a softmax rule
        # in log this looks like this
        
        p_action = np.exp(beta*V)/np.sum(np.exp(beta*V))
        
        # pick an action according to these probabilities
        chosen = np.random.choice([0,1,2,3], size=None, replace=True, p=p_action)
    
        # now we want to learn from feedback on this trial
        # win or lose?
        outcome = np.random.choice([1,0], size=None, replace=True, p=[p_win[chosen], 1-p_win[chosen]])
        
        if outcome:
            V[chosen] += etaP*(win_amount[chosen] - V[chosen])
            # this is the same as writing
            # V[chosen option] = V[chosen option] + eta*(reward - V[chosen option])
        else:
            V[chosen] += etaN*(-pun_dur[chosen] - V[chosen])
        
        # values decay with time if unchosen
        ind = np.setdiff1d([0,1,2,3],chosen)
#         print(ind)
        V[ind] = decay*V[ind]
        
        choice.append(chosen)
        win.append(outcome)
        probs.append(p_action)
        
    return Q,choice,win,probs

# New Simulation Code

In [ ]:
# import random
# random.seed(10)

In [ ]:
# params is a 3 vector of beta, etaP, etaN
params = [2.52937956, 0.02981436, 0.11405636]
ntr = 1000

V = np.zeros(4) # [0,0,0,0]
beta = params[0]
etaP = params[1]
etaN = params[2]
# decay = params[3] # decay between zero and 1
    
p_win = [0.9,0.8,0.5,0.4]
win_amount = [1,2,3,4]
pun_dur = [5,10,30,40]

Q = np.zeros([4,ntr]) #1 array of 4 rows and 1000 columns
choice = []
win = []
probs = []

In [ ]:
Q
arr = np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 16]])
arr[2,:] = V
arr

In [ ]:
for t in range(ntr): #from 0 to 999

    Q[:,t] = V #each column of Q (which represents a t) is now equal to V (4 zeros). This essentially makes each column equal to 4 rows of zero

    p_action = np.exp(beta*V)/np.sum(np.exp(beta*V)) #p_action holds the probabilities of each action (P1-P4), starting at 0.25 for each

    chosen = np.random.choice([0,1,2,3], size=None, replace=True, p=p_action) #a simulated sample of choices (0-3 rep: P1-P4) using the probabilities from p_action
#     print(chosen)
    outcome = np.random.choice([1,0], size=None, replace=True, p=[p_win[chosen], 1-p_win[chosen]]) #a simulated sample of outcomes based on the choice made (in chosen)

    if outcome: #if outcome == 1:
        V[chosen] += etaP*(win_amount[chosen] - V[chosen])
        # this is the same as writing
        # V[chosen option] = V[chosen option] + eta*(reward - V[chosen option])
    else: #if outcome == 0:
        V[chosen] += etaN*(-pun_dur[chosen] - V[chosen])

    choice.append(chosen)
    win.append(outcome)
    probs.append(p_action)

# return Q,choice,win,probs

In [ ]:
len(probs) 

## Goal 3: we now need to acquire actual chosen and outcome data, and ntr... for a certain subject 

In [ ]:
os.chdir("C:\\Users\\dexte\\sparklyRGT\\sparklyRGT_tutorial") 
import sparklyRGT as rgt
os.chdir("C:\\Users\\dexte\\sparklyRGT\\Pystan") 
import model_data as md

In [ ]:
fnames = ['BH09_raw-free_S1-5_corrected.xlsx','CH02_corrected.xlsx','NA01_raw_free-choice_S8-18.xlsx',"CH01_corrected.xlsx"]
df = rgt.load_multiple_data(fnames, reset_sessions = True)

In [ ]:
#rename MSNs so that the rats on the outcome task don't have "loss" in the MSN
for i in range(len(df)):
    if df.at[i, 'MSN'] == 'LossrGT_A-losscue_v1':
        df.at[i,'MSN'] = 'outcomeRGT_A'
    if df.at[i, 'MSN'] == 'LossrGT_B-losscue_v1':
        df.at[i,'MSN'] = 'outcomeRGT_B'
        
#rename MSNs so that the rats on the random task don't have "loss" in the MSN
for i in range(len(df)):
    if df.at[i,'MSN'] == 'AnarchyrGT_B-losscue_v6':
        df.at[i,'MSN'] = 'RandomRGT_B'
    if df.at[i,'MSN'] == 'AnarchyrGT_A-losscue_v6':
        df.at[i,'MSN'] = 'RandomRGT_A'
        
        
task_list = df.groupby(['MSN'])['Subject'].unique()

In [ ]:
uncued_subs = np.concatenate(task_list[[task for task in df.MSN.unique() if 'Classic' in task]])
standard_subs = np.concatenate((task_list['rGT_A-cue'], task_list['rGT_B-cue']))
#concatenating together MisRGT tasks, and RevRGT tasks, as they both refer to reverse-cue RGT
reverse_subs = np.concatenate((np.concatenate(task_list[[task for task in df.MSN.unique() if 'Mis' in task]]),
                              np.concatenate(task_list[[task for task in df.MSN.unique() if 'Rev' in task]])))
outcome_subs = np.concatenate(task_list[[task for task in df.MSN.unique() if 'outcome' in task]])
random_subs = np.concatenate(task_list[[task for task in df.MSN.unique() if 'Random' in task]])
loss_subs = np.concatenate(task_list[[task for task in df.MSN.unique() if 'oss' in task]])

subs = [uncued_subs,standard_subs,reverse_subs,outcome_subs,random_subs,loss_subs]

In [ ]:
uncued_subs

In [ ]:
numsessions = 5 #***first 5 sessions?
uncued = md.get_model_data(df, numsessions, subs[0])

In [ ]:
uncued_df = pd.DataFrame.from_dict(uncued)
uncued_df 
# C (chosen hole of 5), R (reward in pellets, 0 if loss), P (pun_dur, >0 if loss), O (P1-P4 option)

In [ ]:
subjects = uncued_df.startSubject.unique()
np.delete(subjects, 1)

In [ ]:
uncued_df = uncued_df.drop(uncued_df[(uncued_df.C == 0) & (uncued_df.startSubject == 0)].index)
uncued_df.reset_index(inplace = True)
uncued_df.index

In [ ]:
for row in uncued_df.index:
    if uncued_df.at[row,'R'] == 0: #loss
        uncued_df.at[row,'outcome'] = 0
    if uncued_df.at[row,'P'] == 0: #win
        uncued_df.at[row,'outcome'] = 1
    
uncued_df

In [ ]:
uncued_df.index[uncued_df['startSubject'] == 2].tolist()
value = uncued_df.index[uncued_df['startSubject'] == 2]
value

In [ ]:
uncued_1 = uncued_df[0:849] #df for subject 1 
uncued_1[['O', 'outcome']]
chosen_data = uncued_1['O']
outcome_data = uncued_1['outcome']
outcome_data[0]

## Goal 4: after acquiring chosen_data (option, O) and outcome_data (0 or 1 depending on R and P), we now run the model on the data to acquire p_action for each trial

In [ ]:
ntr = 509
for t in range(ntr): #from 0 to 508

    Q[:,t] = V #each column of Q (which represents a t) is now equal to V (4 zeros). This essentially makes each column equal to 4 rows of zero

    p_action = np.exp(beta*V)/np.sum(np.exp(beta*V)) #p_action holds the probabilities of each action (P1-P4), starting at 0.25 for each

    if outcome_data[t] == 1: #if outcome == 1:
        V[chosen_data[t]-1] += etaP*(win_amount[chosen_data[t]-1] - V[chosen_data[t]-1]) #subtract 1 because chosen_data stores P1-P4 as 1-4 instead 0-3
    else: #if outcome_data == 0:
        V[chosen_data[t]-1] += etaN*(-pun_dur[chosen_data[t]-1] - V[chosen_data[t]-1])

    probs.append(p_action)

In [ ]:
#probs

## Goal 5: we now want the log(p_action) for the option chosen. 

ex. p_action = [0.245, 0.28, 0.235, 0.24] ... and P4 was then chosen. We would take log(0.24) for this trial 

In [ ]:
len(probs)
# probs #list of 509 arrays

In [ ]:
len(chosen_data)
# chosen_data #series object of 509 choices (1-4 representing P1-P4)

In [ ]:
import math
math.log10(10)
probs[0][0]
math.log10(probs[0][0]) #first array, first value
log_lik = pd.DataFrame()
log_lik
math.log10(probs[0][chosen_data[1]-1])

In [ ]:
d = {'col1': [0]}
log_lik_test = pd.DataFrame(d)
log_lik_test
log_lik_test[f"log_lik[{0}]"] = math.log10(probs[0][chosen_data[1]-1])
log_lik_test
log_lik = pd.DataFrame(d)

# log_lik_test.assign(log_lik_test=[92,81,66])

In [ ]:
for t in range(len(probs)): 
    log_lik[f"log_lik[{t}]"] = math.log10(probs[t][chosen_data[t]-1])
    
log_lik['sum'] = log_lik.sum(axis = 1)
log_lik

## Goal 6: we now want each column (ex. log_lik[5]) to contain 1000 values (each value is a log likelihood) 

Procedure: 
- Figure out how to add a value to the next row of the same column (ex. add the next value to log_lik[0] as another row)
    - either add a list as a new row
    - or, add a single value to the bottom of a column
- Form list comprehension of trials and params 

In [ ]:
d = {'col1': np.zeros(3)}
log_lik_test = pd.DataFrame(d)
log_lik_test.loc[log_lik_test.index[2], 'col1'] = 4
# df.loc[df.index[someRowNumber], 'New Column Title'] = "some value"
log_lik_test

In [ ]:
ntr = 849

paramsXtrial = [(p, t) for p in range(4000) for t in range(ntr)]
# paramsXtrial

# for (p, t) in paramsXtrial: #one set of parameters, for all the trials, then the next set of parameters (params[1]) for all the trials, etc. 
#     print(p, t)

## Final goal: summarize the code (skip goal 4 and 5) 
- subject 1 of uncued_subs

In [ ]:
# params is a 3 vector of beta, etaP, etaN
import math
params = params_all_4 #1000 sets of parameters

V = np.zeros(4) # [0,0,0,0]
# decay = params[3] # decay between zero and 1
    
p_win = [0.9,0.8,0.5,0.4]
win_amount = [1,2,3,4]
pun_dur = [5,10,30,40]
ntr = 849

Q = np.zeros([4,ntr]) #1 array of 4 rows and ntr columns
probs = []

d = {'col1': np.zeros(1000)}
log_lik = pd.DataFrame(d)

for (p, t) in paramsXtrial: #from 0 to 508

    Q[:,t] = V #each column of Q (which represents a t) is now equal to V (4 zeros). This essentially makes each column equal to 4 rows of zero
    
    if t == 0: #if using a new set of parameters (if first trial, reset)...
        V = np.zeros(4) #reset the V values

    p_action = np.exp(params[p][0]*V)/np.sum(np.exp(params[p][0]*V)) #p_action holds the probabilities of each action (P1-P4), starting at 0.25 for each

    if outcome_data[t] == 1: #if outcome == 1:
        V[chosen_data[t]-1] += params[p][1]*(win_amount[chosen_data[t]-1] - V[chosen_data[t]-1]) #subtract 1 because chosen_data stores P1-P4 as 1-4 instead 0-3
    else: #if outcome_data == 0:
        V[chosen_data[t]-1] += params[p][2]*(-pun_dur[chosen_data[t]-1] - V[chosen_data[t]-1])
         
    log_lik.loc[log_lik.index[p], f"log_lik[{t}]"] = math.log10(p_action[chosen_data[t]-1]) #may want to change to natural log (ln) 
    
    probs.append(p_action)

In [ ]:
log_lik

## Wrap-up: write function(s)
- remove col1 
- transform a sum column, or take the sum of the entire dataframe (log_lik) 

In [ ]:
# probs should be real probabilities

# Q,choice,win,probs = mysimulation([2,0.1,0.02,0.9],ntr)

In [ ]:
fig,ax = plt.subplots(1,2,figsize=[12,5])

ax[0].plot(np.array(probs))
ax[0].legend(['1','2','3','4'])

ax[1].plot(np.arange(ntr),Q.transpose())

In [ ]:
plt.hist(choice)

In [ ]:
# let's check out what softmax does to choice probabilties for diff values

def softmax(x,beta):
    
    return np.exp(beta*x)/np.sum(np.exp(beta*x))

In [ ]:
plt.plot([10,40],softmax(np.array([10,40]),0),'*')
plt.plot([10,40],softmax(np.array([10,40]),0.5),'*')
plt.plot([10,40],softmax(np.array([10,40]),0.01),'*')

Cleanup code

In [ ]:
log_liks = log_lik.drop('col1', axis=1)

In [ ]:
log_liks

In [ ]:
log_liks['row sum'] = log_liks.sum(axis = 1) #watch out! if you run this cell multiple times, it will continue adding

In [ ]:
pw_log_lik = log_liks['row sum'].sum()

In [ ]:
pw_log_lik

### Function (loglik_basic_sub) 

- parameters 
    - outcome_data
    - chosen_data 
    - params 
    - sub

- function template 
    - set variables (V, Q) 
    - set model (p_win, win_amount, pun_dur) 
    - 3 helper functions: 
        - get_outcome_data 
        - get_chosen_data
        - get_ntr_sub
    - run each ntr (should be a range from starting trial to end trial) X sub combination through the model 
    - output: sum of the log-likelihoods, for all subjects!
    
Workflow: 
- create df with subject, range(ntr), chosen_data, outcome_data as the columns 
- create df for each subject (by calling data in a row) 

In [ ]:
def md_subs(df, numsessions, subs): 
    """gets md_df and unique subs (without subject 0)"""
    md_dict = md.get_model_data(df, numsessions, subs) #model data as a dict
    md_df = pd.DataFrame.from_dict(md_dict) #model data as a df
#     md_df = md_df.drop(md_df[(md_df.C == 0) & (md_df.startSubject == 0)].index) #remove non-responses, while keeping changes in startSubject
#     md_df.reset_index(inplace = True)
    
    subs_unique = md_df.startSubject.unique()
    subs_unique = np.delete(subs_unique, 1) #remove subject 0 (not a real subject)
    
    return md_df, subs_unique

In [ ]:
md_df, subs_unique = md_subs(df, 5, standard_subs)

In [ ]:
md_df

In [ ]:
def ntr_sub(md_df, subs_unique): 
    """ntr_sub takes in md_df, and gets the range of trials for a given subject within subs"""
    
    ntr_sub_ranges = []
    ntr_df = pd.DataFrame({'subject#': subs_unique})
    
    #get range for sub: 
    for s in subs_unique: 
        first_t = md_df.index[md_df['startSubject'] == s].tolist() #first trial
        last_t = md_df.index[md_df['startSubject'] == (s+1)].tolist() #last trial
        
        if last_t != []: #if s+1 does not exist (when s = final subject#), last_t = []
            ntr_sub = list(range(first_t[0], last_t[0]))
            ntr_sub_ranges.append(ntr_sub)
        elif last_t == []:
            last_t = [len(md_df)] #last_t = length of md_df
            ntr_sub = list(range(first_t[0], last_t[0]))
            ntr_sub_ranges.append(ntr_sub)
        
    ntr_df['range_ntr'] = ntr_sub_ranges
    return ntr_df #outputs df with subject, and range_ntr as the next column

In [ ]:
sub_ranges = ntr_sub(df, 5, [202, 218, 222, 310, 314, 401, 403, 405, 407, 415, 204, 220, 224, 309, 313, 402, 404, 406, 408, 416, 325, 329, 326, 330])

In [ ]:
sub_ranges

In [ ]:
def chosen_outcome_data(md_df, ntr_df, subs_unique): 
    """takes in md_df, subs_unique and gets chosen and outcome data"""
    
    for row in md_df.index: #add outcome column (where loss == 0, and win == 1)
        if md_df.at[row,'R'] == 0: #loss
            md_df.at[row,'outcome'] = 0
        if md_df.at[row,'P'] == 0: #win
            md_df.at[row,'outcome'] = 1
    
    chosen = []
    outcome = []
    ntr_df = ntr_sub(md_df, subs_unique)
    
    for row in ntr_df.index:
        range_ts = ntr_df.at[row, 'range_ntr'] #range of trials
        filtered_md_df = md_df[range_ts[0]:(range_ts[-1]+1)]
        chosen.append(filtered_md_df['O'].values)
        outcome.append(filtered_md_df['outcome'].values)
                               
    return chosen, outcome

In [ ]:
def get_params_basic_4_chains(fit, subs_unique):
    """takes in fit, subs_unique (list of 1, final sub#), and outputs params_s using ALL CHAINS
    function is called basic due to the parameters chosen"""
    
    fit_beta_df = fit.posterior.beta.to_dataframe()
    fit_etaP_df = fit.posterior.etaPositive.to_dataframe()
    fit_etaN_df = fit.posterior.etaNegative.to_dataframe()
    
    params_s = []
    
    for s in subs_unique:
        params = np.concatenate([fit_beta_df[fit_beta_df.index.get_level_values('beta_dim_0').isin([s-1])],
                                  fit_etaP_df[fit_etaP_df.index.get_level_values('etaPositive_dim_0').isin([s-1])],
                                  fit_etaN_df[fit_etaN_df.index.get_level_values('etaNegative_dim_0').isin([s-1])]], 
                                  axis = 1)
        params_s.append(params)
    return params_s

In [ ]:
chosen1, outcome1 = chosen_outcome_data(df, 5, [202, 218, 222, 310, 314, 401, 403, 405, 407, 415, 204, 220, 224, 309, 313, 402, 404, 406, 408, 416, 325, 329, 326, 330])

In [ ]:
def sub_df_complete(df, numsessions, subs, fit):
    """takes the output from md_subs, ntr_sub and chosen_outcome_data functions and outputs the complete df"""
    md_df, subs_unique = md_subs(df, numsessions, subs)
    ntr_df = ntr_sub(md_df, subs_unique)
    chosen_data, outcome_data = chosen_outcome_data(md_df, ntr_df, subs_unique)
    params_s = get_params_basic_4_chains(fit, subs_unique)
    
    ntr_df['chosen_data'] = chosen_data
    ntr_df['outcome_data'] = outcome_data
    ntr_df['params_s'] = params_s
    
    return ntr_df, md_df

In [ ]:
sub_df, md_df = sub_df_complete(df, 5, standard_subs, standard_basic_fit)

In [ ]:
sub_df

In [ ]:
md_df

Does len(chosen_data) match range_ntr
There's 0's in chosen_data - resolved

# Building new dataset

In [ ]:
uncued_basic_fit

In [ ]:
uncued_4_chains = uncued_basic_fit.to_dataframe()
uncued_4_chains
uncued_1_chain = uncued_4_chains.iloc[0:1000]
uncued_1_chain
uncued_1_chain_no_index = uncued_1_chain.reset_index(drop = True)
uncued_1_chain_no_index

In [ ]:
log_liks = log_lik.drop('col1', axis=1)
log_liks

In [ ]:
uncued_1_chain_no_index.iloc[:,[0,1]]

In [ ]:
df_concat = pd.concat([uncued_1_chain_no_index.iloc[:,[0,1]], log_liks], axis = 1)
df_concat #df_concat is the complete dataframe for subject 1

In [ ]:
df_concat = df_concat.set_index(["chain", "draw"])
x_concat = xr.Dataset.from_dataframe(df_concat)
x_concat

In [ ]:
dataset_concat = az.InferenceData(sample_stats=x_concat)
dataset_concat

In [ ]:
# az.waic(dataset_concat, var_name = 'log_lik')

# add df as an xarray to inferenceData

In [ ]:
log_liks = log_lik.drop('col1', axis = 1)

In [ ]:
log_liks

Xarray example

In [ ]:
x = xr.Dataset(
    {
        "temperature_c": (
            ("lat", "lon"),
            20 * np.random.rand(4).reshape(2, 2),
        ),
        "precipitation": (("lat", "lon"), np.random.rand(4).reshape(2, 2)),
    },
    coords={"lat": [10, 20], "lon": [150, 160]},

In [ ]:
x.assign(temperature_f=lambda x: x.temperature_c * 9 / 5 + 32)

In [ ]:
x.assign(temperature_f=x["temperature_c"] * 9 / 5 + 32)

In [ ]:
temperature_f=x["temperature_c"] * 9 / 5 + 32

In [ ]:
temperature_f

to_xarray example

In [ ]:
df = pd.DataFrame([('falcon', 'bird', 389.0, 2),
                   ('parrot', 'bird', 24.0, 2),
                   ('lion', 'mammal', 80.5, 4),
                   ('monkey', 'mammal', np.nan, 4)],
                  columns=['name', 'class', 'max_speed',
                           'num_legs'])
df

In [ ]:
df.to_xarray()

In [ ]:
df['max_speed'].to_xarray()

In [ ]:
log_liks_dataset = log_liks.to_xarray()

In [ ]:
uncued_basic_fit_log_lik = uncued_basic_fit.assign(log_liks_dataset)

In [ ]:
uncued_basic_fit_log_lik

In [ ]:
az.waic(uncued_basic_fit_log_lik, var_name = log_lik)

Centered eight example of az.waic

data = az.load_arviz_data("centered_eight")

In [ ]:
data

In [ ]:
data.sample_stats.log_likelihood.to_dataframe()

indices are chain, draw, trial
and the values are log_likelihood values

In [ ]:
log_liks

Multiindex example --> completed with log_likelihood added to standard_basic_fit

In [ ]:
ntr = sub_df.at[1, 'range_ntr']
ntr[-1]
# ntr[-1] - ntr[0] # 445
# list(range(ntr[-1]+1 - ntr[0]))

In [ ]:
paramsXtrial = [(p, t) for p in range(4000) for t in range(ntr[-1] - ntr[0])]

In [ ]:
def get_log_lik_values_s(sub_df):
    """takes in sub_df, and gets all the log_likelihood values in a list"""
    
    log_lik_values = [] #stores log_lik_values for all subjects
    for s in sub_df.index: #note: s is 0-indexed
        
        #subject data
        ntr = sub_df.at[s,'range_ntr']
        chosen_data = sub_df.at[s,'chosen_data']
        outcome_data = sub_df.at[s,'outcome_data']
        params = sub_df.at[s, 'params_s'] #4000 sets of parameters 
        
        #updating model values
        V = np.zeros(4) # [0,0,0,0]
#         Q = np.zeros([4,ntr]) #1 array of 4 rows and ntr columns
        
        #model
        p_win = [0.9,0.8,0.5,0.4]
        win_amount = [1,2,3,4]
        pun_dur = [5,10,30,40]
        
        paramsXtrial = [(p, t) for p in range(4000) for t in range(ntr[-1]+1 - ntr[0])] #4000 sets of parameters, for ntr trials
        for (p, t) in paramsXtrial: 

            if t == 0: #if using a new set of parameters (if first trial, reset)...
                V = np.zeros(4) #reset the V values

            p_action = np.exp(params[p][0]*V)/np.sum(np.exp(params[p][0]*V)) #p_action holds the probabilities of each action (P1-P4), starting at 0.25 for each

            if outcome_data[t] == 1: 
                V[chosen_data[t]-1] += params[p][1]*(win_amount[chosen_data[t]-1] - V[chosen_data[t]-1]) #subtract 1 because chosen_data stores P1-P4 as 1-4 instead 0-3
            else: 
                V[chosen_data[t]-1] += params[p][2]*(-pun_dur[chosen_data[t]-1] - V[chosen_data[t]-1])
            print(p_action[chosen_data[t]-1])
            log_lik_values.append(math.log(p_action[chosen_data[t]-1])) 
    
    return log_lik_values

In [ ]:
log_lik_values = get_log_lik_values_s(sub_df) #24 minutes

In [ ]:
len(log_lik_values)

In [ ]:
# params is a 3 vector of beta, etaP, etaN
params = params_all_4 #1000 sets of parameters

V = np.zeros(4) # [0,0,0,0]
# decay = params[3] # decay between zero and 1
    
p_win = [0.9,0.8,0.5,0.4]
win_amount = [1,2,3,4]
pun_dur = [5,10,30,40]
ntr = 849

Q = np.zeros([4,ntr]) #1 array of 4 rows and ntr columns
probs = []

s = []
for (p, t) in paramsXtrial: #from 0 to 508
    
    if t == 0: #if using a new set of parameters (if first trial, reset)...
        V = np.zeros(4) #reset the V values

    p_action = np.exp(params[p][0]*V)/np.sum(np.exp(params[p][0]*V)) #p_action holds the probabilities of each action (P1-P4), starting at 0.25 for each

    if outcome_data[t] == 1: #if outcome == 1:
        V[chosen_data[t]-1] += params[p][1]*(win_amount[chosen_data[t]-1] - V[chosen_data[t]-1]) #subtract 1 because chosen_data stores P1-P4 as 1-4 instead 0-3
    else: #if outcome_data == 0:
        V[chosen_data[t]-1] += params[p][2]*(-pun_dur[chosen_data[t]-1] - V[chosen_data[t]-1])
         
    s.append(math.log10(p_action[chosen_data[t]-1])) #may want to change to natural log (ln) 

In [ ]:
log_lik_values = s 

In [ ]:
len(log_lik_values)

Add log_lik_values to MultiIndex (chain, draw, trial) 

In [ ]:
iterables = [[0,1,2,3], list(range(0,1000)), list(range(0, 25091))] #4 chains, 1000 draws, ntr trials
index = pd.MultiIndex.from_product(iterables, names=["chain", "draw", "trial"])
log_lik_s = pd.Series(log_lik_values, index=index)
df_log_likelihood = log_lik_s.to_frame(name = "log_likelihood")
x_log_likelihood = df_log_likelihood.to_xarray()

id_log_likelihood = standard_basic_fit.assign(x_log_likelihood)
id_log_likelihood #2 minutes

In [ ]:
def get_inference_data_log_lik(md_df, log_lik_values, fit):
    """takes in log_lik_values from get_log_lik_values, and creates an inference data object"""
    
    total_ntr = md_df.at[0, 'ntr'] #total trials in md_df, which represents the total number of trials for a cue variant 
    print(total_ntr)
    print(4000)
    print(len(log_lik_values))
    
    #build index
    iterables = [[0,1,2,3], list(range(0,1000)), list(range(0, total_ntr))] #4 chains, 1000 draws, total_ntr trials
    index = pd.MultiIndex.from_product(iterables, names=["chain", "draw", "trial"])
    
    #build InferenceData (id) object 
    s_log_likelihood = pd.Series(log_lik_values, index=index)
    df_log_likelihood = s_log_likelihood.to_frame(name = "log_likelihood")
    x_log_likelihood = df_log_likelihood.to_xarray()
    id_log_likelihood = fit.assign(x_log_likelihood)
    
    return id_log_likelihood

In [ ]:
get_inference_data_log_lik(md_df, log_lik_values, standard_basic_fit)

In [ ]:
def waic_basic_fit(df, numsessions, subs, fit):
    """summary function: takes in all parameters and outputs WAIC table using az.waic() // parameters:
    df - product of rgt.load_multiple_data,
    numsessions = 5,
    subs = cue variant (list), 
    fit = .nc object storing the model and fit"""
    
    sub_df, md_df = sub_df_complete(df, numsessions, subs, fit)
    log_lik_values = get_log_lik_values_s(sub_df)
    id_log_likelihood = get_inference_data_log_lik(md_df, log_lik_values, fit)
    
    waic = az.waic(id_log_likelihood)
    return waic